In [ ]:
import json
import random

# Simulaciones de Herramientas de consulta y razonamiento
def clasificacion(query):
    # Herramienta de Razonamiento: Logica para identificar la intencion
    if "reprogramar" in query or "cambiar hora" in query:
        return {"categoria": "reprogramacion_cita", "intencion": "cambiar_cita"}
    elif "horario" in query or "visita" in query:
        return {"categoria": "pregunta_frecuente", "intencion": "consultar_horario_visita"}
    elif "reclamo" in query:
        return {"categoria": "reclamo", "intencion": "registrar_reclamo"}
    return {"categoria": "otra_consulta"}

def contexto(query, knowledge_base):
    # Herramienta de Consulta: Simulacion del Pipeline RAG (Recuperacion Semantica)
    if "visita" in query or "horario" in query:
        return knowledge_base["horarios_visita"]
    elif "medicamento" in query:
        return knowledge_base["farmacia_disponibilidad"]
    elif "dolor" in query or "urgencias" in query:
        return knowledge_base["urgencias"]
    return None

def generador_respuesta(query, context, memory_buffer):
    # Herramienta de Escritura: Genera la respuesta usando contexto y memoria
    
    # Logica de coherencia de memoria para flujos de reclamo
    is_in_reclamo_flow = any("reclamo" in entry for entry in memory_buffer[-2:])
    
    if is_in_reclamo_flow and "problema" in query:
        return "Entendido. Su problema ha sido registrado como parte del reclamo escalado. ¿Necesita algo mas?"
    
    # Uso del Contexto Sematico (RAG)
    if context and "horario" in context:
        return "El horario de visita es de 15:00 a 17:00 de lunes a viernes."
    elif context and "farmacia" in context:
        return "Puede consultar la disponibilidad de su medicamento en el portal web del hospital, ya que el inventario se actualiza constantemente."
    elif context and "urgencias" in context:
        return "Nuestros protocolos de urgencia sugieren que ante un dolor intenso es mejor dirigirse a la sala de emergencias. Le sugiero ir a la brevedad."
    
    # Respuesta por defecto si no hay contexto RAG
    return "Disculpe, no tengo una respuesta precisa para su consulta. Por favor, reformule su pregunta."

class HospitalAIAgent:
    def __init__(self):
        # Memoria de Largo Plazo: Base de Conocimiento para el RAG
        self.knowledge_base = {
            "urgencias": "Nuestros protocolos de urgencia indican que ante un dolor intenso...",
            "horarios_visita": "El horario de visita es de 15:00 a 17:00 de lunes a viernes.",
            "farmacia_disponibilidad": "La farmacia central tiene un inventario actualizado. La disponibilidad de farmacos se puede consultar en el portal web...",
            "protocolo_reclamos": "Para registrar un reclamo, necesitamos la fecha del incidente y el servicio afectado."
        }
        # Memoria de Corto Plazo: Buffer de conversacion
        self.memory_buffer = []

    def update_memory(self, user_query, agent_response):
        # Actualiza el historial de conversacion
        self.memory_buffer.append(f"Usuario: {user_query}")
        self.memory_buffer.append(f"Agente: {agent_response}")
        self.memory_buffer = self.memory_buffer[-6:] # Limitar la memoria a 3 turnos
        
    # Orquestacion y Planificacion (IE5, IE6)
    def process_query(self, user_query):
        # 1. Razonamiento: Clasificar (IE1)
        classification = clasificacion(user_query)
        category = classification.get("categoria", "otra_consulta")
        derivation_needed = False
        
        # 2. Consulta: Recuperacion de Contexto Sematico (RAG) (IE4)
        context_data = contexto(user_query, self.knowledge_base)
        
        # 3. Planificacion y Toma de Decisiones (IE5, IE6)
        if category == "reprogramacion_cita":
            # Decision: Flujo automatizado de alta prioridad
            final_response = "Hemos iniciado el proceso de reprogramacion. Un agente se contactara."
        
        elif category == "reclamo":
            # Decision: Escalamiento obligatorio para garantizar trazabilidad
            final_response = "Para registrar formalmente su reclamo, necesito sus datos de contacto y la fecha del incidente. Un funcionario recibira la informacion directamente."
            derivation_needed = True # Decision adaptativa: Derivar
        
        elif context_data:
            # Decision: Responder Automaticamente si el RAG fue exitoso (pregunta_frecuente o urgencia)
            final_response = generador_respuesta(user_query, context_data, self.memory_buffer)
        
        else: # 'otra_consulta' y no hay contexto RAG
            # Decision: Derivacion por complejidad o falta de informacion en la Base de Conocimiento
            final_response = "Lo siento, su consulta es compleja o no esta en mi base de datos. ¿Le gustaria ser atendido por un funcionario especializado?"
            derivation_needed = True # Decision adaptativa: Derivar
        
        # 4. Memoria: Actualizar el Buffer (IE3)
        self.update_memory(user_query, final_response)

        return {
            "query": user_query,
            "response": final_response,
            "derivation_to_human": derivation_needed,
            "current_memory": self.memory_buffer
        }

if __name__ == "__main__":
    agent = HospitalAIAgent()
    
    # --- PRUEBAS DE TOMA DE DECISIONES (IE6) ---
    
    # Prueba 1: Decision de Respuesta Automatica (RAG Exitoso)
    q1 = "¿Cual es el horario de visita?" 
    print(f"\n--- Prueba 1: RAG Exitoso (Decision: Responder Auto) ---")
    response_data = agent.process_query(q1)
    print(f"Consulta: {response_data['query']}")
    print(f"Respuesta: {response_data['response']}")
    print(f"Derivacion Humana: {'Si' if response_data['derivation_to_human'] else 'No'} (Decision correcta)")

    # Prueba 2: Decision de Escalamiento Inmediato (Reclamo)
    q2 = "Tengo un reclamo por un problema en la atencion."
    print(f"\n--- Prueba 2: Reclamo (Decision: Derivar Inmediato) ---")
    response_data = agent.process_query(q2)
    print(f"Consulta: {response_data['query']}")
    print(f"Respuesta: {response_data['response']}")
    print(f"Derivacion Humana: {'Si' if response_data['derivation_to_human'] else 'No'} (Decision adaptativa por politica)")

    # Prueba 3: Decision de Respuesta por Protocolo de Urgencia (RAG Exitoso en otra categoria)
    q3 = "Mi estómago me duele mucho, debo pedir hora o ir a urgencias?"
    print(f"\n--- Prueba 3: Urgencia (Decision: Responder Protocolo RAG) ---")
    response_data = agent.process_query(q3)
    print(f"Consulta: {response_data['query']}")
    print(f"Respuesta: {response_data['response']}")
    print(f"Derivacion Humana: {'Si' if response_data['derivation_to_human'] else 'No'} (Decision adaptativa por seguridad)")
    
    # Prueba 4: Decision de Derivacion por Complejidad (RAG Fallido o no aplica)
    q4 = "Necesito saber el nombre del medico que me atendio hace tres años."
    print(f"\n--- Prueba 4: Consulta Compleja/Sin Contexto (Decision: Derivar) ---")
    response_data = agent.process_query(q4)
    print(f"Consulta: {response_data['query']}")
    print(f"Respuesta: {response_data['response']}")
    print(f"Derivacion Humana: {'Si' if response_data['derivation_to_human'] else 'No'} (Decision adaptativa por limite de conocimiento)")

>> Inicializando la base de conocimiento...

>> Procesando la consulta: 'Quiero saber si tienen el medicamento para el dolor de cabeza.'
   ->  Clasificacion: Categoria 'otra_consulta'
   -> La consulta es compleja. Activando el RAG.
   -> Buscando contexto relevante...
   -> Generando respuesta con el contexto...

--- Resultado Final del Sistema ---
Consulta del Paciente: Quiero saber si tienen el medicamento para el dolor de cabeza.
Respuesta del Sistema: Disculpe, no tengo una respuesta precisa para su consulta. Por favor, reformule su pregunta.
¿Necesita derivacion humana? No
-----------------------------------

>> Procesando la consulta: 'Como puedo cambiar la hora que tengo con el doctor?'
   ->  Clasificacion: Categoria 'otra_consulta'
   -> La consulta es compleja. Activando el RAG.
   -> Buscando contexto relevante...

--- Resultado Final del Sistema ---
Consulta del Paciente: Como puedo cambiar la hora que tengo con el doctor?
Respuesta del Sistema: Lo siento, no tengo sufici